# Revisiting the Primer Vector

In this notebook we revisit the primer vector theory from Lawden from the lens of first order variations.

## Theory and notation

------------

a) Consider the following definition of the State Transition Matrix, $\mathbf M_{fs}$:

$$
\mathbf M_{fs} := \frac{\partial \mathbf x_{f}}{\partial \mathbf x_{s}}
$$

Note how this definition does not depend on the dynamics. The STM allows to describe variations of the (final) state at $f$ as:

$$
\delta\mathbf x_f = \mathbf M_{fs} \delta \mathbf x_s
$$

We also make use of the following definitions for the various blocks of the STM:

$$
\mathbf M = \left[ 
\begin{array}{c|c} 
  \mathbf M^{rr} & \mathbf M^{rv} \\ 
  \hline 
  \mathbf M^{vr} & \mathbf M^{vv} 
\end{array} 
\right] 
= \left[ 
\begin{array}{c|c} 
  \mathbf M^{xr} & \mathbf M^{xv} 
\end{array} 
\right] 
= \left[ 
\begin{array}{c} 
  \mathbf M^{rx} \\
  \hline 
  \mathbf M^{vx} 
\end{array} 
\right] 
$$

---------------

b) Assume now to have a grid of $N$ points along a multiple impulse trajectory and pick three indexes $i,j,k$. 

We ask the following question: what happens when we add three small $\delta\Delta V$ at the selected nodes?

To answer this question, we compute the variation of the final state due to intermidiate variations at the nodes using the STMs:

$$
\delta \mathbf x_f = \mathbf M_{fi}\delta\mathbf x_i + \mathbf M_{fj}\delta\mathbf x_j + \mathbf M_{fk}\delta\mathbf x_k
$$

which we set to zero, as we do not want the trajectory boundary cnstraints to change, only find a better $\Delta V$:

$$
\mathbf 0 = \mathbf M_{fi}\delta\mathbf x_i + \mathbf M_{fj}\delta\mathbf x_j + \mathbf M_{fk}\delta\mathbf x_k
$$

which becomes (multiplying by $\mathbf M_{jf}$):

$$
\mathbf M_{ji}\delta\mathbf x_i + \mathbf M_{jk}\delta\mathbf x_k + \delta\mathbf x_j = \mathbf 0
$$

--------------

c) The state variations $\delta\mathbf x$ are, in our case, consequence of three $\delta\Delta \mathbf V$, so that the previous equations becomes:

$$
\begin{align}
\mathbf M_i^{rv} \delta\Delta \mathbf V_i + \mathbf M_k^{rv} \delta\Delta \mathbf V_k = \mathbf 0\\
\mathbf M_i^{vv} \delta\Delta \mathbf V_i + \mathbf M_k^{vv} \delta\Delta \mathbf V_k + \delta\Delta \mathbf V_j = \mathbf 0\\
\end{align}
$$

which becomes:

$$
\begin{align}
\delta\Delta \mathbf V_i &=& - (\mathbf M_i^{rv})^{-1}\mathbf M_k^{rv} \delta\Delta \mathbf V_k = \mathbf A_{ik}\delta\Delta \mathbf V_k\\
\delta\Delta \mathbf V_j &=& - \big(\mathbf M_i^{vv}\mathbf A_{ik} + \mathbf M_k^{vv} \big)  \delta\Delta \mathbf V_k = \mathbf A_{jk}\delta\Delta \mathbf V_k\\
\end{align}
$$

The matrices $\mathbf A$ are telling us how the three variations of impulsive velocity changes applied in ($i$, $j$, $k$) must be related for the overall trajectory to not change its boundary conditions (i.e. $\mathbf x_f = \mathbf 0$).

-------------

d) So far the three indexes we picked (i.e. $i,j,k$) were completely equivalent, now we assume that in $i,j$ a finite $\Delta V$ is already present, while in $k$ only our additional $\delta\Delta \mathbf V$ will be present.

The total magnitude of the $\Delta \mathbf V$ can then be expressed by:

$$
J = \Delta V_{tot} = |\mathbf V_i + \delta\Delta \mathbf V_i | + |\mathbf V_j + \delta\Delta \mathbf V_j | + |\delta\Delta \mathbf V_k|
$$

and its first order variation:

$$
\delta J = \frac{\Delta\mathbf V_i}{|\Delta \mathbf V_i|}\cdot |\delta\Delta \mathbf V_i| + \frac{\Delta \mathbf V_j}{|\Delta \mathbf V_j|}\cdot |\delta\Delta \mathbf V_j| + |\delta\Delta \mathbf V_k|
$$

e) The primer vector

We introduce $\hat{\mathbf u} = \frac{\delta \Delta \mathbf V_k}{|\delta \Delta \mathbf V_k|}$ as the unit vector along the direction of the $\delta \Delta V$ added in $k$. Substituting and regrouping we have:

$$
\delta J = |\delta\Delta \mathbf V_k| \left(1 + \left(\mathbf A_{ik}^T\frac{\Delta\mathbf V_i}{|\Delta \mathbf V_i|} + \mathbf A_{jk}^T\frac{\Delta\mathbf V_j}{|\Delta \mathbf V_j|} \right) \cdot \hat {\mathbf u}\right)
$$

which we rewrite as:

$$
\delta J = |\delta\Delta \mathbf V_k| \left(1 - \mathbf p\cdot \hat {\mathbf u}\right)
$$

where we have introduced the vector:

$$
\boxed{\mathbf p = - \mathbf A_{ik}^T\frac{\Delta\mathbf V_i}{|\Delta \mathbf V_i|} - \mathbf A_{jk}^T\frac{\Delta\mathbf V_j}{|\Delta \mathbf V_j|}}
$$

which is of fundamental importance in space flight mechanics and is called **primer vector**. 

:::{note}
From the expression of $\delta J$ it appears clear that if we want to be able to decrease the overall $\Delta V$ adding in $k$ one impulse it is necessary for the norm of the primer vector (computed in $k$) to be larger than 1.

Classically the result of the primer vector is derived starting from Pontryagin maximum principle, here we have presented an original derivation building on the work from Bauregard et al. publication {cite:p}`beauregard`, which will allow to extend also the primer vector to new, previously untreated, cases (see the notebook [A primer vector surrogate](<./primer_vector_surrogate.ipynb>)).

Enough of that, let us start coding now ...

In [2]:
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt

Let us define a problem. In the Keplerian dynamics, we construct an arbitrary two impulse trajectory

In [3]:
# Problem data
n_points = 200
# Initial position
r0 = np.array([1., 0., 0.])
v0 = np.array([0., 1., 0.])
# Magnitude of the DVs
DVi = np.array([0.4, -0.2, -0.1])
DVj = np.array([-0.02, 0.2, 0.21])
# time grid
t_grid = np.linspace(0, 4*np.pi, n_points)

# Nodes at which the impulses are given
idx_i = 9
idx_j = 19

We first compute the trajectory as well as the STMs. We need to propagate at each impulse, add the impulse, then proceed.

In [4]:
retval0 = pk.propagate_lagrangian_grid([r0, v0], t_grid[:idx_i+1], mu=1, stm=True)
ri, vi = retval0[-1][0]
vi = [a+b for a,b in zip(vi, DVi)]
retval1 = pk.propagate_lagrangian_grid([ri, vi], t_grid[idx_i+1: idx_j+1], mu=1, stm=True)
rj, vj = retval1[-1][0]
vj = [a+b for a,b in zip(vj, DVj)]
retval2 = pk.propagate_lagrangian_grid([rj, vj], t_grid[idx_j+1:], mu=1, stm=True)

The various STMs thus computed are (for example) M00, M10, M20 | M22, M32, M42| M44, Mf4. Instead we want M00, M10, M20, M30, M40, Mf0. 

In [5]:
retval1.pop(0);

In [8]:
pk.propagate_lagrangian_grid([ri, vi], [0, 0.1], mu=1, stm=False)[1][0]

[[0.8248796077843502, 0.5997678310828135, -0.009983856518866123],
 [-0.2191338758490723, 0.5876258856698118, -0.09952044776053727]]

In [9]:
pk.propagate_lagrangian([ri, vi], 0.1, mu=1, stm=False)

([0.8248796077843502, 0.5997678310828135, -0.009983856518866123],
 [-0.2191338758490723, 0.5876258856698118, -0.09952044776053727])